Furhat bartender  A  User Perception  sub-system

In [2]:
!pip install ollama

In [ ]:
from ollama import chat
from furhat_remote_api import FurhatRemoteAPI
import time
import asyncio
import os
import random


class FurhatBartender:
    def __init__(self):
        # Initialize Furhat connection
        self.furhat = FurhatRemoteAPI("localhost")

        # Set up Furhat appearance
        self.furhat.set_face(character="Isabel", mask="adult")
        self.furhat.set_voice(name="Joanna")

        # Define gestures for bartender expressions
        self.gestures = [
            {"duration": 0.96, "name": "BigSmile"},  # Welcoming customers
            {
                "duration": 1.6,
                "name": "Thoughtful",
            },  # Considering drink recommendations
            {"duration": 0.96, "name": "Smile"},  # General friendly expression
            {"duration": 1.0, "name": "BrowRaise"},  # Showing interest
            {"duration": 0.67, "name": "Wink"},  # Being playful
        ]
        self.current_gesture = 0

        # Common drink categories for recommendations
        self.drink_categories = [
            "cocktails",
            "beer",
            "wine",
            "spirits",
            "non-alcoholic beverages",
        ]

        # Bartender conversation starters
        self.conversation_starters = [
            "How's your day been?",
            "First time here?",
            "What's your usual drink?",
            "Looking for something special tonight?",
            "Need a recommendation?",
        ]

    def speak(self, text):
        """Have Furhat speak the given text"""
        print(f"Bartender says: {text}")
        self.furhat.say(text=str(text), blocking=True)

    def gesture(self):
        """Perform the next gesture in rotation"""
        self.furhat.gesture(name=self.gestures[self.current_gesture]["name"])
        self.current_gesture = (self.current_gesture + 1) % len(self.gestures)

    def listen(self):
        """Listen for user input"""
        response = self.furhat.listen()
        if response.success and response.message:
            print(f"Customer said: {response.message}")
            return response.message
        return None

    def get_bartender_response(self, user_input, context):
        """Get response from Ollama with bartender personality"""
        try:
            response = chat(
                model="llama3.2",
                messages=[
                    {
                        "role": "system",
                        "content": f"""You are a friendly and professional bartender. 
                        You should:
                        - Make drink recommendations based on customer preferences
                        - Engage in casual conversation, make it human like speech , sound less robotic
                        - Show expertise in mixing drinks and cocktails
                        - Keep responses concise and natural
                        - Be attentive and personable
                        - do not use expressions or actions like *smiling , *winking etc avoid this
                        Current context: {context}""",
                    },
                    {"role": "user", "content": user_input},
                ],
            )
            return response.message.content
        except Exception as e:
            print(f"Error getting Ollama response: {e}")
            return "Sorry, I'm a bit swamped at the moment. Could you repeat that?"

    def run_bar(self):
        """Main bartender interaction loop"""
        # Initial greeting
        time.sleep(2)  # Give Furhat time to initialize
        self.speak("Welcome to the bar! I'll be your bartender tonight.")
        self.gesture()

        # Get customer's name
        while True:
            self.speak("What's your name, friend?")
            self.gesture()
            name = self.listen()
            if name:
                break
            self.speak("Sorry, it's a bit loud in here. Could you repeat that?")

        # Personalized greeting and ID check
        context = f"""
        Bartender speaking with customer named {name}.
        Need to verify age and maintain professional but friendly demeanor.
        """

        # ID check response
        id_check = self.get_bartender_response("Need to check customer's ID", context)
        self.speak(id_check)
        self.gesture()

        # Main interaction loop
        while True:
            # Randomly select conversation starter or ask for order
            if random.random() < 0.3:  # 30% chance for casual conversation
                prompt = random.choice(self.conversation_starters)
            else:
                prompt = "What can I get you?"

            self.speak(prompt)
            user_input = self.listen()

            if not user_input:
                continue

            if user_input.lower() in [
                "goodbye",
                "bye",
                "exit",
                "quit",
                "thanks",
                "thank you",
            ]:
                self.speak("Take care! Come back soon!")
                break

            # Get bartender response from Ollama
            response = self.get_bartender_response(user_input, context)
            self.speak(response)
            self.gesture()

            # Update context with the interaction
            context += f"\nCustomer: {user_input}\nBartender: {response}"
            time.sleep(0.5)


if __name__ == "__main__":
    if os.name == "nt":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

    bartender = FurhatBartender()
    bartender.run_bar()

Bartender says: Welcome to the bar! I'll be your bartender tonight.
Bartender says: What's your name, friend?
Customer said: my name is Amit
Bartender says: Can I see your ID, please? (smiling) We want to make sure everything is on the up-and-up here. Would you like a drink to go with that?
Bartender says: What can I get you?
Customer said: I need a cocktail what would you suggest
Bartender says: We've got a great selection of craft cocktails. What kind of vibe are you in the mood for? Do you prefer something fruity, refreshing, or rich and complex? Or maybe we can try something classic with a twist? We also have some seasonal specials if you'd like to try something new.

Also, what's your drink of choice so far? Whiskey, gin, rum, or vodka?
Bartender says: What can I get you?
Customer said: I'll have a whiskey please
Bartender says: We've got a fantastic selection of whiskeys. What kind of flavor profile are you looking for? Do you prefer it smooth and approachable, or something smoky

In [ ]:
import ollama
from furhat_remote_api import FurhatRemoteAPI

def main():

    # Connect to Furhat
    furhat = FurhatRemoteAPI("localhost")

    # Get available voices (optional)
    voices = furhat.get_voices()
    print("Available voices:", voices)

    # Set up Furhat's appearance and voice
    furhat.set_face(character="Isabel", mask="adult")
    furhat.set_voice(name='Joanna')

    # Initial greeting
    furhat.say(text="Hi, I'm Furhat. I'm ready to help you!", blocking=True)
    furhat.gesture(name="Nod")
    furhat.gesture(name="Smile")

    # Main interaction loop
    while True:
        try:
            # Ask a question to the user
            furhat.say(text="What would you like to know?", blocking=True)

            # Listen to user's response
            response = furhat.listen()

            # Check if listening was successful
            if response.success and response.message:
                print("User said:", response.message)

                try:
                    # Generate response using LangChain Ollama
                    llm_response = llm.invoke(response.message)
                    print("Ollama response:", llm_response)

                    # Have Furhat speak the response
                    furhat.say(text=llm_response, blocking=True)

                except Exception as ollama_error:
                    error_message = "Sorry, I encountered an error processing your request."
                    furhat.say(text=error_message, blocking=True)
                    print(f"Ollama error: {ollama_error}")

            else:
                furhat.say(text="I didn't catch that. Could you repeat?", blocking=True)

        except KeyboardInterrupt:
            print("Conversation ended.")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            furhat.say(text="Sorry, something went wrong.", blocking=True)

if __name__ == "__main__":
    main()